In [1]:
import pandas as pd
import numpy as np
import time
import pymysql
from sshtunnel import SSHTunnelForwarder

In [2]:
mysql_configure = pd.read_csv("/Volumes/Lab_Public/Yuan/temp/mysql_connection.csv",index_col=0)

In [3]:
sql_hostname = mysql_configure.loc["sql_hostname",]["value"]
sql_username = mysql_configure.loc["sql_username",]["value"]
sql_password = mysql_configure.loc["sql_password",]["value"]
sql_main_database = mysql_configure.loc["sql_main_database",]["value"]
sql_port = mysql_configure.loc["sql_port",]["value"]
ssh_host = mysql_configure.loc["ssh_host",]["value"]
ssh_user = mysql_configure.loc["ssh_user",]["value"]
ssh_password = mysql_configure.loc["ssh_password",]["value"]
ssh_port = mysql_configure.loc["ssh_port",]["value"]

In [6]:
with SSHTunnelForwarder(
        (ssh_host, int(ssh_port)),
        ssh_username=ssh_user,
        ssh_password=ssh_password,
        remote_bind_address=('127.0.0.1', int(sql_port))) as tunnel:
    print('SSH connected')
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.Distance_angle da, pdbdb.Site s "
                sql_where = "WHERE da.pdbID = s.pdbID and da.chain = s.chain and da.ID_1=s.Seq and da.Res_1 = \"HIS\";"    
                sql = sql_select+sql_where
                data = pd.read_sql_query(sql, conn)

    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()


SSH connected


In [82]:
sql

'Select da.* FROM pdbdb.Distance_angle da, pdbdb.Site s WHERE (da.pdbID = s.pdbID) and (da.chain = s.chain) and (da.ID_1=s.Seq) ;'

In [7]:
data.shape

(9441, 9)